# DATA 608 Assignment 5: What Is The Effect Of The Earth's Temperature on Cyclonic Storms?

Historic data indicates that the occurrence and intensity of cyclonic storms (Hurricanes, Typhoons and Tornados) increases with the increased earth temperature. For this assignment you will need to tell this story to a non-technical audience (eg: a high-school earth science class)

In [3]:
# load dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns

# Data Preparation

In [14]:
# load the data set for earth anomalies
anomalies_url = 'https://raw.githubusercontent.com/eddiexunyc/earth_temp_cyclone_data/refs/heads/main/Resources/anomalies_data.csv'
anomalies_data = pd.read_csv(anomalies_url)

# load the hurricane data set and cleanse it
hur_data_url = 'https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2023-051124.txt'
hurricane_raw_data = pd.read_csv(hur_data_url, sep = ',').reset_index()
hurricane_raw_data = hurricane_raw_data.drop(['AL011851', '            UNNAMED', '     14', 'Unnamed: 3','level_7',
                                              'level_8', 'level_9', 'level_10', 'level_11', 'level_12', 
                                              'level_13', 'level_14', 'level_15', 'level_16'], axis=1)

hurricane_raw_data

,level_0,level_1,level_2,level_3,level_4,level_5,level_6
0,18510625,0000,,HU,28.0N,94.8W,80.0
1,18510625,0600,,HU,28.0N,95.4W,80.0
2,18510625,1200,,HU,28.0N,96.0W,80.0
3,18510625,1800,,HU,28.1N,96.5W,80.0
4,18510625,2100,L,HU,28.2N,96.8W,80.0
...,...,...,...,...,...,...,...
56716,20231023,1800,,TD,11.5N,83.2W,25.0
56717,20231024,0000,,TD,12.2N,83.4W,25.0
56718,20231024,0130,L,TD,12.4N,83.5W,25.0
56719,20231024,0600,,TD,13.0N,83.8W,25.0
